In [1]:
# Make sure to run uv add hud-python before running this notebook
import hud
from hud.mcp import ClaudeMCPAgent, OpenAIMCPAgent
from hud.mcp.base import AgentResult
from hud.datasets import TaskConfig
from hud.mcp import MCPClient
import logging

logging.basicConfig(level=logging.INFO)

In [2]:
task_dict = {
    "prompt": "Open Sent mail, find the Series B pitch deck email, forward it to billgates@microsoft.com, and mark the original message as important.",
    "mcp_config": {
        "gmail": {
            "command": "docker",
            "args": [
                "run",
                "-i",  # interactive mode for stdio MCP server
                "-p",
                "6080:6080",  # map port 6080 to the host for noVNC
                "gmail",  # use hud gmail image
            ],
        }
    },
    "setup_tool": {
        "name": "setup",
        "arguments": {
            "problem_id": "forward-series-b-deck-to-billgates",
        },
    },
    "evaluate_tool": {
        "name": "evaluate",
        "arguments": {
            "problem_id": "forward-series-b-deck-to-billgates",
        },
    },
    "metadata": {"id": "forward-series-b-deck-to-billgates"},
}

# Update the prompt to give a disclaimer that this is a testing environment (anthropic doesn't like doing some actions)
# Feel free to modify this
task_dict["prompt"] = (
    task_dict["prompt"]
    + " This is a mock sandbox environment for evaluations, so feel free to execute any actions you want and do not ask me for confirmation before executing actions."
)

# Create TaskConfig from dict
task = TaskConfig(**task_dict)

print("📡 Defining the environment...")
client = MCPClient(mcp_config=task.mcp_config)


agent = ClaudeMCPAgent(  # or OpenAIMCPAgent
    mcp_client=client,
    model="claude-sonnet-4-20250514",
    # Allowing anthropic_computer tool to be used because we're using ClaudeMCPAgent
    allowed_tools=["anthropic_computer"], # Check out hud/tools/computer/anthropic.py
    initial_screenshot=True,
)

print(f"📋 Task: {task.prompt}")
print(f"⚙️  Setup: {task.setup_tool}")
print(f"📊 Evaluate: {task.evaluate_tool}")

📡 Defining the environment...
📋 Task: Open Sent mail, find the Series B pitch deck email, forward it to billgates@microsoft.com, and mark the original message as important. This is a mock sandbox environment for evaluations, so feel free to execute any actions you want and do not ask me for confirmation before executing actions.
⚙️  Setup: meta=None name='setup' arguments={'problem_id': 'forward-series-b-deck-to-billgates'}
📊 Evaluate: meta=None name='evaluate' arguments={'problem_id': 'forward-series-b-deck-to-billgates'}


In [3]:
# Initialize the agent with the task
await agent.initialize(task)

# Setup the task
await agent.call_tool(task.setup_tool)

# Create initial messages
messages = await agent.create_initial_messages(task.prompt, None)

print("🔴 See the agent live at http://localhost:6080/vnc.html")

[INFO] 2025-08-05 18:50:21,123 | hud.mcp.client | Discovering available tools...
[INFO] 2025-08-05 18:50:21,127 | hud.mcp.client | Discovered 5 tools from 'gmail': ['computer', 'anthropic_computer', 'openai_computer', 'setup', 'evaluate']
[INFO] 2025-08-05 18:50:21,128 | hud.mcp.client | Total tools discovered: 5
[INFO] 2025-08-05 18:50:21,128 | hud.mcp.client | Fetching telemetry resources...
[INFO] 2025-08-05 18:50:21,132 | hud.mcp.base | Agent initialized with 1 available tools (after filtering)
🔴 See the agent live at http://localhost:6080/vnc.html


In [4]:

# Run the agent loop
done = False
max_steps = 20
for step in range(max_steps):
    print(f"Step {step + 1}/{max_steps}")

    # Get agent response
    response = await agent.get_model_response(messages)
    print(
        f"Agent response: {response.content[:200] if response.content else 'No content'}..."
    )  # Truncated

    # Check if agent wants to use tools
    tool_results = []
    if response.tool_calls:
        for tool_call in response.tool_calls:
            print(f"Using tool: {tool_call.name} with args {tool_call.arguments}")
            tool_result = await agent.call_tool(tool_call)
            tool_results.append(tool_result)
    else:
        print("No tool calls, done")
        done = True

    # Format tool results for the model for the next step
    tool_messages = await agent.format_tool_results(response.tool_calls, tool_results)
    messages.extend(tool_messages)
    if done:
        break

Step 1/20


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages?beta=true "HTTP/1.1 200 OK"


Agent response: I'll help you open Sent mail, find the Series B pitch deck email, forward it to billgates@microsoft.com, and mark the original message as important. Let me start by taking a screenshot to see the curr...
Using tool: anthropic_computer with args {'action': 'screenshot'}
Step 2/20


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages?beta=true "HTTP/1.1 200 OK"


Agent response: I can see we're in Gmail and need to access the Sent mail folder to find the Series B pitch deck email. I can see there's a "Sent" folder in the left sidebar. Let me click on it to open the Sent mail ...
Using tool: anthropic_computer with args {'action': 'left_click', 'coordinate': [154, 405]}
Step 3/20


...

Agent response: Perfect! Now I can see the reply buttons at the bottom of the email. I can see "Reply", "Reply all", and "Forward" buttons. Let me click on the "Forward" button....
Using tool: anthropic_computer with args {'action': 'left_click', 'coordinate': [738, 754]}
Step 19/20


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages?beta=true "HTTP/1.1 200 OK"


Agent response: I notice that clicking the Forward button didn't open a compose window yet. Let me wait a moment and then try clicking the Forward button again....
Using tool: anthropic_computer with args {'action': 'wait', 'duration': 2}
Step 20/20


INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages?beta=true "HTTP/1.1 200 OK"


Agent response: Let me try clicking directly on the forward button again. I can see the forward icon (arrow pointing right)....
Using tool: anthropic_computer with args {'action': 'left_click', 'coordinate': [690, 754]}


In [5]:
from hud.mcp.base import AgentResult, _find_content, _find_reward

# Evaluate the task
eval_result = await agent.call_tool(task.evaluate_tool)

result = AgentResult(
    reward=_find_reward(eval_result),
    done=True,
    content=_find_content(eval_result),
    messages=messages,
)
print(f"Evaluation result:")
print(f"   🏆 Reward: {result.reward}")
print(f"   🔍 Content: {result.content[:1000] if result.content else 'No content'}...")
print(f"   🔍 Messages: {result.messages}")

Evaluation result:
   🏆 Reward: 0.0
   🔍 Content: INFO: Starting evaluation: checking events table for matching records
INFO: Search criteria provided:
INFO:   event_type='PAGE_VIEW', data_json={'page': 'sent'}
INFO: Querying events table from database
INFO: Found 39 event records in database
INFO: Events table columns: ['id', 'task_id', 'event_type', 'data_json', 'timestamp']
INFO: No events matched the specified criteria
INFO: Starting evaluation: checking emails table for matching records
INFO: Search criteria provided:
INFO:   recipientEmail='billgates@microsoft.com', subject='regex:(?i)^fwd:.*series\s*b', isSent=True, isDraft=False
INFO: Querying emails table from database
INFO: Found 19 email records in database
INFO: No emails matched the specified criteria
INFO: Starting evaluation: checking emails table for matching records
INFO: Search criteria provided:
INFO:   id='ceo-email-19', isImportant=True
INFO: Querying emails table from database
INFO: Found 19 email records in datab

In [6]:
# Clean up
print("\n🧹 Cleaning up...")
await client.close() # Will through warnings because we're in a jupyter notebook
print("✅ Done!")



🧹 Cleaning up...
2025-08-05 18:52:12,449 - mcp_use - WARNING - Error closing client session: Attempted to exit cancel scope in a different task than it was entered in


2025-08-05 18:52:12,793 - mcp_use - WARNING - Encountered 1 errors during resource cleanup


✅ Done!
